In [ ]:
import dask.dataframe as dd
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import boto3
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error,average_precision_score,mean_squared_error
import joblib
import os
import seaborn as sns
from bayes_opt import BayesianOptimization
import lightgbm as lgb
import warnings
from sklearn.preprocessing import label_binarize
from multiprocessing import  cpu_count
from catboost import CatBoostClassifier,Pool
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from lifetimes import ModifiedBetaGeoFitter
from lifetimes import GammaGammaFitter
from dateutil.relativedelta import relativedelta 
from lifetimes.utils import summary_data_from_transaction_data
import re
from unicodedata import normalize
# from shared import util
import gc
from sklearn.utils import class_weight
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline
import os
import time

# import sys
import sys
sys.path.append(os.path.dirname(os.path.expanduser(".")))
sys.path.append(os.path.dirname(os.path.expanduser("..")))
sys.path.append(os.path.dirname(os.path.expanduser("../defines")))
from defines import *

import boto3
s3 = boto3.client('s3')
s3.download_file("fury-data-apps", "marketing-utils/pzenone/utils.py","utils.py")
import utils

target_col = "frequency_eval"

from preprocessing import generate_summary_cal,reg_data,frequencies,ASP,SI,Ordenes,prepaid,franquero,pareto,IPT,IPT_short,IPS,locations,month_data_size,month_data_sales,month_data_recency,visitas_short, demograficos, compras, calificaciones, sellers, payments, payments_all, mp_buyer, mp_seller, movimientos_wallet,asset_mgm,target,long_term_data
from train_utils import train_combinations,simple_fit,foo_evaluation_regression,foo_model_regression,foo_predict_regression,anti_dummies,caster,picture

In [ ]:
import pymc3 as pm

summary_cal = utils.read_pickle_csv("s3://"+fda_path+s3_path_in+"summary_train_0.pkl")[["recency_date"]].sample(n = 80000)
summary_cal = summary_cal.loc[summary_cal.recency_date < 1000]
summary_eval = utils.read_pickle_csv("s3://"+fda_path+s3_path_in+"summary_test_0.pkl")[["recency_date"]].sample(n = 80000)
summary_eval = summary_eval.loc[summary_eval.recency_date < 1000]

print("paso")
with pm.Model():
    alpha1 = pm.Normal('alpha1', mu=1/7.1, sigma=0.5)
    exp1 = pm.Exponential("exp1", alpha1, observed=summary_cal.recency_date.values)
    
    alpha2 = pm.Normal('alpha2', mu=1/7.1, sigma=0.5)
    exp2 = pm.Exponential("exp2", alpha2, observed=summary_cal.recency_date.values)

    step = pm.Metropolis()
    trace = pm.sample(10000, step=step)
    burned_trace = trace[1000:]

In [ ]:
plt.hist(burned_trace["alpha1"], bins=25, histtype="stepfilled", normed=True,alpha = 0.5,color = 'r')
plt.hist(burned_trace["alpha2"], bins=25, histtype="stepfilled", normed=True,alpha = 0.5,color = 'g')